In [9]:
import numpy as np
random = np.random.default_rng()
a=random.random(size=1)[0]
a

0.6927511236630035

In [10]:
import numpy as np
import matplotlib.pyplot as plt
import torch
from queue import Queue
import gym
import math
from copy import deepcopy


import relearn.pies.dqn as DQN
from relearn.explore import EXP, MEM
from relearn.pies.utils import compare_weights
from relearn.pies.utils import RMSprop_update

random = np.random.default_rng()

In [11]:
class INFRA:
    """ Dummy empty class"""

    def __init__(self):
        pass

EXP_PARAMS = INFRA()
EXP_PARAMS.MEM_CAP = 50000
EXP_PARAMS.EPST = (0.95, 0.05, 0.95)  # (start, min, max)
EXP_PARAMS.DECAY_MUL = 0.99999
EXP_PARAMS.DECAY_ADD = 0


PIE_PARAMS = INFRA()
PIE_PARAMS.LAYERS = [6, 6, 6]
PIE_PARAMS.OPTIM = torch.optim.RMSprop  # SGD
PIE_PARAMS.LOSS = torch.nn.MSELoss
PIE_PARAMS.LR = 0.001
PIE_PARAMS.DISCOUNT = 0.999999
PIE_PARAMS.DOUBLE = False
PIE_PARAMS.TUF = 4
PIE_PARAMS.DEV = 'cpu'

TRAIN_PARAMS = INFRA()
TRAIN_PARAMS.EPOCHS = 50000
TRAIN_PARAMS.MOVES = 10
TRAIN_PARAMS.EPISODIC = False
TRAIN_PARAMS.MIN_MEM = 30
TRAIN_PARAMS.LEARN_STEPS = 1
TRAIN_PARAMS.BATCH_SIZE = 50
TRAIN_PARAMS.TEST_FREQ = 10

TEST_PARAMS = INFRA()
TEST_PARAMS.CERF = 100
TEST_PARAMS.RERF = 100


P = print


In [12]:
ENV_NAME = 'CartPole-v0'
# Train ENV
env = gym.make(ENV_NAME)

# Test ENV
venv = gym.make(ENV_NAME)

# Policy and Exploration
exp = EXP(env=env, cap=EXP_PARAMS.MEM_CAP, epsilonT=EXP_PARAMS.EPST)

txp = EXP(env=venv, cap=math.inf, epsilonT=(0, 0, 0))
eps = []

def decayF(epsilon, moves, isdone):
    global eps
    new_epsilon = epsilon*EXP_PARAMS.DECAY_MUL + \
        EXP_PARAMS.DECAY_ADD  # random.random()
    eps.append(new_epsilon)
    return new_epsilon


pie = DQN.PIE(
    env.observation_space.shape[0],
    LL=PIE_PARAMS.LAYERS,
    action_dim=env.action_space.n,
    device=PIE_PARAMS.DEV,
    opt=PIE_PARAMS.OPTIM,
    cost=PIE_PARAMS.LOSS,
    lr=PIE_PARAMS.LR,
    dis=PIE_PARAMS.DISCOUNT,
    mapper=lambda x: x,
    double=PIE_PARAMS.DOUBLE,
    tuf=PIE_PARAMS.TUF,
    seed=None)

target = DQN.PIE(
    env.observation_space.shape[0],
    LL=PIE_PARAMS.LAYERS,
    action_dim=env.action_space.n,
    device=PIE_PARAMS.DEV,
    opt=PIE_PARAMS.OPTIM,
    cost=PIE_PARAMS.LOSS,
    lr=PIE_PARAMS.LR,
    dis=PIE_PARAMS.DISCOUNT,
    mapper=lambda x: x,
    double=PIE_PARAMS.DOUBLE,
    tuf=PIE_PARAMS.TUF,
    seed=None)


In [18]:
params= pie.Q.state_dict()

In [19]:
print(params)

OrderedDict([('SEQL.0.weight', tensor([[-0.2866,  0.3710,  0.3079,  0.1787],
        [ 0.0843,  0.3274,  0.4275,  0.2642],
        [ 0.2556,  0.0432,  0.1439, -0.1493],
        [-0.3140, -0.2874, -0.2890, -0.1802],
        [-0.1336,  0.1896, -0.4590, -0.2334],
        [ 0.0506,  0.1840, -0.1863,  0.4687]])), ('SEQL.0.bias', tensor([-0.0398,  0.3610, -0.4434, -0.1778,  0.0875,  0.4760])), ('SEQL.2.weight', tensor([[ 0.0578, -0.2902,  0.1470, -0.3694,  0.0189,  0.1525],
        [-0.3207, -0.1140,  0.0210, -0.3528,  0.1873,  0.0803],
        [ 0.1039,  0.2874, -0.1917, -0.3206,  0.0997, -0.2694],
        [-0.0655, -0.3877,  0.1973, -0.1922, -0.0627,  0.3935],
        [ 0.1191,  0.2580, -0.0547, -0.0931, -0.4057,  0.0754],
        [ 0.2676, -0.0806,  0.1748, -0.0999, -0.0420, -0.3469]])), ('SEQL.2.bias', tensor([ 0.3829, -0.3070,  0.0887, -0.1854,  0.3643,  0.2443])), ('SEQL.4.weight', tensor([[-0.2784, -0.1385, -0.0169,  0.1968,  0.2554, -0.3388],
        [-0.0066,  0.1968, -0.3824, -0.24

In [21]:
params_ = {}
for key in params.keys():
    params_[key] = params[key].tolist()
    print(params[key].shape)
    print(params[key], "R: ", torch.rand(params[key].shape))

torch.Size([6, 4])
tensor([[-0.2866,  0.3710,  0.3079,  0.1787],
        [ 0.0843,  0.3274,  0.4275,  0.2642],
        [ 0.2556,  0.0432,  0.1439, -0.1493],
        [-0.3140, -0.2874, -0.2890, -0.1802],
        [-0.1336,  0.1896, -0.4590, -0.2334],
        [ 0.0506,  0.1840, -0.1863,  0.4687]]) R tensor([[0.9410, 0.3609, 0.2451, 0.6570],
        [0.2134, 0.8430, 0.5392, 0.1140],
        [0.0755, 0.1691, 0.2210, 0.6337],
        [0.8642, 0.5881, 0.2194, 0.8937],
        [0.6084, 0.4247, 0.6506, 0.9542],
        [0.1145, 0.0390, 0.0192, 0.0732]])
torch.Size([6])
tensor([-0.0398,  0.3610, -0.4434, -0.1778,  0.0875,  0.4760]) R tensor([0.6655, 0.3111, 0.5242, 0.4190, 0.1516, 0.9249])
torch.Size([6, 6])
tensor([[ 0.0578, -0.2902,  0.1470, -0.3694,  0.0189,  0.1525],
        [-0.3207, -0.1140,  0.0210, -0.3528,  0.1873,  0.0803],
        [ 0.1039,  0.2874, -0.1917, -0.3206,  0.0997, -0.2694],
        [-0.0655, -0.3877,  0.1973, -0.1922, -0.0627,  0.3935],
        [ 0.1191,  0.2580, -0.0547, 